In [14]:
import torch
import os

class Para:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    data_path = 'data/cs_space_20d_rate_20d_2017_pre'

    month_num = len(os.listdir(data_path))

para = Para()
print(para.month_num)

60


In [15]:
import numpy as np
import torch
from my_utils.model_class import MLP
import pandas as pd

def insert_predict(df: pd.DataFrame, model_path, out_nums: int, column_name: str) -> pd.DataFrame:

    # 加载模型
    model = MLP(in_nums=22, out_nums=out_nums, drop_p=0.5)
    model.load_state_dict(torch.load(model_path, map_location=para.device))
    model = model.to(para.device)

    # 运算
    model.eval()
    predict_tensor = model(torch.Tensor(df.loc[:, 'ep_ratio_ttm': 'BR'].values).to(device=para.device))

    # 插入 predict_tensor 列
    df.insert(loc=0, column='predict_tensor_'+column_name, value=np.nan)
    # 插入 predict_return_bin 列
    df.insert(loc=0, column='predict_return_bin_'+column_name, value=np.nan)

    df['predict_tensor'+'_'+column_name] = predict_tensor.to(device='cpu').detach().numpy().tolist()
    df['predict_return_bin'+'_'+column_name] = predict_tensor.to(device='cpu').detach().numpy().argmax(1)

    return df

In [16]:
import torchmetrics
import time
import matplotlib.pyplot as plt

time_begin = time.time()

precision = torchmetrics.Precision(average='none', num_classes=2)

for i_csv in range(para.month_num):
# for i_csv in range(0, 1):
    print()
    print('i_csv: {}'.format(i_csv))
    print('use time: {}'.format(time.time() - time_begin))

    df = pd.read_csv(para.data_path +'/' + str(i_csv) + '.csv', index_col='order_book_id')

    # df = insert_predict(df=df, model_path='models/model_c2_p[0.2, 0.2]_y5_w[1.0, 1.0]_b32_lr0.001_d0.5.pth', out_nums=2, column_name='2c0')
    # df = insert_predict(df=df, model_path='models/model_c2_p[0.05, 0.05]_y5_w[1.0, 1.0]_b32_lr0.001_d0.5.pth', out_nums=2, column_name='2c1')
    # df = insert_predict(df=df, model_path='models/model_c2_p[0.01, 0.01]_y5_w[1.0, 1.0]_b32_lr0.001_d0.5.pth', out_nums=2, column_name='2c2')
    df = insert_predict(df=df, model_path='models/model_c2_b32_lr0.001_d0.5.pth', out_nums=2, column_name='2c3')
    # 插入三分类预测数据
    # df = insert_predict(df=df, model_path='models/model_c3_p[0.2, 0.2, 0.2]_y5_w[1.0, 1.0, 1.0]_b32_lr0.001_d0.5.pth', out_nums=3, column_name='3c0')

    # 综合多个分类预测数据
    df.insert(loc=0, column='predict_return_bin', value=np.nan)

    # df.loc[df['predict_return_bin_2c1'].isin([0,]), 'predict_return_bin'] = 0
    # df.loc[df['predict_return_bin_3c0'].isin([0,]) & df['predict_return_bin_2c1'].isin([0,]), 'predict_return_bin'] = 0
    # df.loc[df['predict_return_bin_3c0'].isin([0,]), 'predict_return_bin'] = 0
    df.loc[df['predict_return_bin_2c3'].isin([0,]), 'predict_return_bin'] = 0

    df['predict_return_bin'].replace(to_replace=np.nan, value=1, inplace=True)
    df['predict_return_bin'] = df['predict_return_bin'].astype('int64')

    # 插入实际 return_bin 列
    df.insert(loc=0, column='return_bin', value=np.nan)
    df.loc[df['yield_rate'] > 0.0, 'return_bin'] = 0
    # df.iloc[0:int(df.shape[0]*0.01), df.columns.get_loc('return_bin')] = 0

    df['return_bin'].replace(to_replace=np.nan, value=1, inplace=True)
    df['return_bin'] = df['return_bin'].astype('int64')

    # 计算精确率
    df = df.sort_values(by='yield_rate', ascending=False)
    pred = torch.tensor(df['predict_return_bin']).type(torch.LongTensor)
    target = torch.tensor(df['return_bin']).type(torch.LongTensor)
    print('date: {}' .format(df['date'][0]))
    print(precision(preds=pred, target=target))

    print(df.loc[df['yield_rate']>0.2,'predict_return_bin'].value_counts())
    print(df.loc[df['yield_rate']<-0.2,'predict_return_bin'].value_counts())


i_csv: 0
use time: 0.016709089279174805
date: 2017-01-03
tensor([0.4889, 0.6035])
0    681
1    238
Name: predict_return_bin, dtype: int64
0    707
1    404
Name: predict_return_bin, dtype: int64

i_csv: 1
use time: 0.29399871826171875
date: 2017-02-07
tensor([0.5034, 0.6231])
0    682
1    230
Name: predict_return_bin, dtype: int64
0    635
1    391
Name: predict_return_bin, dtype: int64

i_csv: 2
use time: 0.35648345947265625
date: 2017-03-07
tensor([0.4275, 0.5969])
0    577
1    256
Name: predict_return_bin, dtype: int64
0    810
1    408
Name: predict_return_bin, dtype: int64

i_csv: 3
use time: 0.4304049015045166
date: 2017-04-06
tensor([0.4842, 0.5945])
0    721
1    228
Name: predict_return_bin, dtype: int64
0    748
1    371
Name: predict_return_bin, dtype: int64

i_csv: 4
use time: 0.5241310596466064
date: 2017-05-05
tensor([0.5224, 0.5665])
0    749
1    271
Name: predict_return_bin, dtype: int64
0    670
1    383
Name: predict_return_bin, dtype: int64

i_csv: 5
use time: 0

In [17]:

print()
print('average precision: {}' .format(precision.compute()))


average precision: tensor([0.4453, 0.5975])


## 测试模型预测

In [18]:
import torch
from my_utils.model_class import MLP

model = MLP(in_nums=22, out_nums=2, drop_p=0.5)
model.load_state_dict(torch.load(para.model_path, map_location=para.device))
model = model.to(para.device)

model.eval()
a = model(torch.Tensor([[0.5]*22]*4).to(device=para.device))
a

AttributeError: 'Para' object has no attribute 'model_path'